# dependencies
# !pip install h5py

In [34]:
import pandas as pd
import os

def has_header(file_path, expected_columns=None):
    """
    Check if a CSV file has a header row by examining the first row's values.
    """

    # Read only the first row
    df = pd.read_csv(file_path, nrows=1)
    first_row = df.columns.tolist()
    
    # If expected_columns is provided, check for case-insensitive match
    if expected_columns:
        return all(col.lower() in [e.lower() for e in expected_columns] for col in first_row)
    
    # Otherwise, assume header if all values are strings and not numeric
    return all(isinstance(col, str) and not col.replace('.', '', 1).isdigit() for col in first_row)

def is_empty(file_path):
    # Check if file is empty by size
    if os.path.getsize(file_path) == 0:
        return True
    
    # Try to read the CSV to check for data rows
    try:
        df = pd.read_csv(file_path, nrows=1)
        return df.empty or len(pd.read_csv(file_path)) == 0
    except pd.errors.EmptyDataError:
        return True  # File is empty or invalid
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return True  # Treat other errors as empty to avoid crashing

def show_head(file_path):
    df = pd.read_csv(file_path)
    # Print first 3 rows
    print(df.head(3))



In [ ]:
btc_file = '/Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT/BTCUSD_Daily_OHLC.csv'
#is_empty(btc_file)
# Warning: some files are empty
CELOEUR_1 = '/Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT/CELOEUR_1.csv'
is_empty(CELOEUR_1)
show_head(CELOEUR_1) # empty

In [4]:
# THIS IS THE ONLY file with headers
file = '/Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT/BTCUSD_Daily_OHLC.csv'
has_header(file)

True

### Create Summary CSV


In [35]:
import os
import pandas as pd
from datetime import datetime, timezone


# Specify the folder containing the CSV files
folder_path = '/Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT'

# Expected columns in the CSV files
expected_columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'trades']
expected_num_columns = len(expected_columns)

# List to store information about each file
file_info_list = []

# Iterate over all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):

        file_path = os.path.join(folder_path, file_name)
        # Print empty files and exclude them
        if is_empty(file_path):
            print(f"Warning: {file_name} is empty")
            continue

        # Read the CSV file, with or without header
        if has_header(file_path):
            df = pd.read_csv(file_path)
        else:
            df = pd.read_csv(file_path, header=None, names=expected_columns)
        
        # 1. Check if the file has the expected 7 columns
        if len(df.columns) != expected_num_columns or not all(col in expected_columns for col in df.columns):
            print(f"Warning: {file_name} does not have the expected {expected_num_columns} columns: {df.columns.tolist()}")
            continue
        
        # 2. Extract first and last timestamps
        first_unix_time = df['timestamp'].iloc[0]
        last_unix_time = df['timestamp'].iloc[-1]
        
        # Convert timestamps to human-readable UTC dates
        # Use timezone.utc instead of UTC
        first_date = datetime.fromtimestamp(first_unix_time, timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        last_date = datetime.fromtimestamp(last_unix_time, timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        
        number_rows = len(df)
        
        # Store file information
        file_info = {
            'file_name': file_name,
            'first_unix_time': first_unix_time,
            'last_unix_time': last_unix_time,
            'first_date': first_date,
            'last_date': last_date,
            'number_rows': number_rows
        }
        file_info_list.append(file_info)


# Convert file info list to a DataFrame for easy viewing
file_info_df = pd.DataFrame(file_info_list)

# Print the results
print("\nSummary of CSV files:")
print(file_info_df)

# Optionally, save the summary to a CSV file
output_path = os.path.join(folder_path, 'csv_file_summary.csv')
file_info_df.to_csv(output_path, index=False)
print(f"\nSummary saved to {output_path}")


Summary of CSV files:
             file_name  first_unix_time  last_unix_time           first_date  \
0         TRUUSD_1.csv       1661529540      1743460500  2022-08-26 15:59:00   
1       MEEUR_1440.csv       1738022400      1743033600  2025-01-28 00:00:00   
2        ARPAUSD_5.csv       1659018900      1743452400  2022-07-28 14:35:00   
3      OCEANGBP_30.csv       1704067200      1727712000  2024-01-01 00:00:00   
4      LAYERUSD_30.csv       1741116600      1743462000  2025-03-04 19:30:00   
...                ...              ...             ...                  ...   
8233  ATLASUSD_720.csv       1643284800      1743422400  2022-01-27 12:00:00   
8234     INJUSD_60.csv       1628607600      1743462000  2021-08-10 15:00:00   
8235   NTRNUSD_240.csv       1721145600      1743451200  2024-07-16 16:00:00   
8236   STGUSD_1440.csv       1664409600      1743379200  2022-09-29 00:00:00   
8237  USDQUSD_1440.csv       1732147200      1743379200  2024-11-21 00:00:00   

                

In [3]:
import os
import pandas as pd
from datetime import datetime, timezone

pd.set_option('display.max_colwidth', None)  # Don't truncate column content
pd.set_option('display.width', 1000)        # Set a large width to fit content on one line

filepath = '/Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT_summary/csv_file_summary.csv'
df = pd.read_csv(filepath)
print(df.head(10))
# df = pd.read_csv(filepath)
# df.sort_values(by='first_unix_time', ascending=True, inplace=True)
# df.to_csv(filepath, index=False


         file_name  first_unix_time  last_unix_time           first_date            last_date  number_rows
0  XBTEUR_1440.csv       1378771200      1743379200  2013-09-10 00:00:00  2025-03-31 00:00:00         4216
1   XBTEUR_720.csv       1378814400      1743422400  2013-09-10 12:00:00  2025-03-31 12:00:00         8412
2    XBTEUR_60.csv       1378854000      1743462000  2013-09-10 23:00:00  2025-03-31 23:00:00       100032
3    XBTEUR_15.csv       1378856700      1743464700  2013-09-10 23:45:00  2025-03-31 23:45:00       395562
4     XBTEUR_5.csv       1378856700      1743465300  2013-09-10 23:45:00  2025-03-31 23:55:00      1152434
5     XBTEUR_1.csv       1378856820      1743465540  2013-09-10 23:47:00  2025-03-31 23:59:00      5172692
6  LTCEUR_1440.csv       1379116800      1743379200  2013-09-14 00:00:00  2025-03-31 00:00:00         4177
7   LTCEUR_720.csv       1379116800      1743422400  2013-09-14 00:00:00  2025-03-31 12:00:00         8336
8    LTCEUR_60.csv       1379145600  

In [42]:
import pandas as pd
import re

# Read the original file
df = pd.read_csv(filepath)

# Define allowed intervals
valid_intervals = {"1", "5", "15", "30", "60", "240", "720", "1440"}

# Function to extract interva (timestep) from filename
def extract_interval(filename):
    match = re.search(r'_(\d+)\.csv$', filename)
    return match.group(1) if match else None

# Apply extraction
df['interval'] = df.iloc[:, 0].apply(extract_interval)

# Split the summary df by interval, create a new summary df for each interval
for interval in valid_intervals:
    df_interval = df[df['interval'] == interval]
    if not df_interval.empty:
        df_interval.drop(columns='interval').to_csv(f"{filepath}_{interval}.csv", index=False)

# Identify and print rows with invalid or missing intervals
invalid_rows = df[~df['interval'].isin(valid_intervals)]
if not invalid_rows.empty:
    print("Rows with unrecognized or missing intervals:")
    print(invalid_rows)


Rows with unrecognized or missing intervals:
                file_name  first_unix_time  last_unix_time           first_date            last_date  number_rows interval
12  BTCUSD_Daily_OHLC.csv       1381017600      1703980800  2013-10-06 00:00:00  2023-12-31 00:00:00         3727     None


In [45]:
import pandas as pd
# Define your base filepath (without _<interval>.csv)
base_filepath = filepath  # e.g., "data/XBTEUR"

# Define valid intervals
intervals = ["1", "5", "15", "30", "60", "240", "720", "1440"]

for nb in intervals:
    file = f"{base_filepath}_{nb}.csv"
    try:
        df = pd.read_csv(file)
        df.sort_values(by='first_unix_time', ascending=True, inplace=True)
        df.to_csv(file, index=False)
        print(f"Length of file {nb}: {len(df)}")
    except FileNotFoundError:
        print(f"{file} not found.")


Length of file 1: 1038
Length of file 5: 1037
Length of file 15: 1037
Length of file 30: 1009
Length of file 60: 1037
Length of file 240: 1009
Length of file 720: 1035
Length of file 1440: 1035


In [20]:
import pandas as pd
import re
import os

def compute_missing_percentage(summary_csv_path):
    # Load the summary CSV
    df = pd.read_csv(summary_csv_path)

    # Parse timestep from filename column
    def extract_timestep(file_name):
        match = re.search(r'_(\d+)\.csv$', file_name)
        return int(match.group(1)) * 60 if match else None  # Convert minutes to seconds

    timestep = df['file_name'].apply(extract_timestep)

    # Compute expected number of rows
    expected_rows = (df['last_unix_time'] - df['first_unix_time']) // timestep

    # Compute percentage missing
    df['perc_missing'] = 100 * (expected_rows - df['number_rows']) / expected_rows

    # Save or return updated DataFrame
    df.to_csv(summary_csv_path, index=False)
    print(f"Updated {summary_csv_path} with 'perc_missing' column.")

    return df


import glob

def batch_process_summary_files(folder_path):
    summary_files = glob.glob(os.path.join(folder_path, 'csv_file_summary.csv_*.csv'))
    for summary_file in summary_files:
        print(f"Processing {summary_file}...")
        compute_missing_percentage(summary_file)
# Example usage:
# compute_missing_percentage("csv_file_summary.csv_60.csv")
batch_process_summary_files('/Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT_summary')

Processing /Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT_summary/csv_file_summary.csv_15.csv...
Updated /Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT_summary/csv_file_summary.csv_15.csv with 'perc_missing' column.
Processing /Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT_summary/csv_file_summary.csv_60.csv...
Updated /Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT_summary/csv_file_summary.csv_60.csv with 'perc_missing' column.
Processing /Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT_summary/csv_file_summary.csv_1440.csv...
Updated /Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT_summary/csv_file_summary.csv_1440.csv with 'perc_missing' column.
Processing /Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/Kraken_OHLCVT_summary/csv_file_summary.csv_5.csv...
Updated /Users/xin/Stanford/MSE 349 Financial Statistics

In [22]:
# Extract files with 'USD_' from summmary, sort by decending 'first_unix_time'

def filter_summary(number: int):
    """
    Sort and summarize USD_ files by first_unix_time in descending order and save to CSV.
    
    Args:
        number: The number to filter files by (e.g. 60 or 720)
    """
    # Read the summary file
    summary_file = f"Kraken_OHLCVT_summary/csv_file_summary.csv_{number}.csv"
    df = pd.read_csv(summary_file)
    
    # Filter for USD_ files and sort by first_unix_time in descending order
    summary = df[df['file_name'].str.contains('USD_')].sort_values('first_unix_time', ascending=False)
    
    # Format the missing percentage to 2 decimal places
    summary['perc_missing'] = summary['perc_missing'].map('{:.2f}%'.format)
    
    # Save to CSV
    output_file = f"Kraken_OHLCVT_summary/USD_{number}_summary.csv"
    summary.to_csv(output_file, index=False)
    
    # Print the summary
    print(f"\nSummary of USD_ files (sorted by start date):")
    print(f"Total number of USD_ files: {len(summary)}")
    print(f"Summary saved to: {output_file}")
    print("\n", summary.to_string(index=False))
    
    return summary

In [23]:
filter_summary(720)


Summary of USD_ files (sorted by start date):
Total number of USD_ files: 389
Summary saved to: Kraken_OHLCVT_summary/USD_720_summary.csv

            file_name  first_unix_time  last_unix_time          first_date           last_date  number_rows perc_missing
     CORNUSD_720.csv       1743120000      1743422400 2025-03-28 00:00:00 2025-03-31 12:00:00            8      -14.29%
     TERMUSD_720.csv       1743076800      1743422400 2025-03-27 12:00:00 2025-03-31 12:00:00            9      -12.50%
   GHIBLIUSD_720.csv       1743076800      1743422400 2025-03-27 12:00:00 2025-03-31 12:00:00            9      -12.50%
      WALUSD_720.csv       1743033600      1743422400 2025-03-27 00:00:00 2025-03-31 12:00:00           10      -11.11%
      NILUSD_720.csv       1742990400      1743422400 2025-03-26 12:00:00 2025-03-31 12:00:00           11      -10.00%
    SPICEUSD_720.csv       1742472000      1743422400 2025-03-20 12:00:00 2025-03-31 12:00:00           23       -4.55%
   APENFTUSD_720.cs

,file_name,first_unix_time,last_unix_time,first_date,last_date,number_rows,perc_missing
1033,CORNUSD_720.csv,1743120000,1743422400,2025-03-28 00:00:00,2025-03-31 12:00:00,8,-14.29%
1031,TERMUSD_720.csv,1743076800,1743422400,2025-03-27 12:00:00,2025-03-31 12:00:00,9,-12.50%
1030,GHIBLIUSD_720.csv,1743076800,1743422400,2025-03-27 12:00:00,2025-03-31 12:00:00,9,-12.50%
1027,WALUSD_720.csv,1743033600,1743422400,2025-03-27 00:00:00,2025-03-31 12:00:00,10,-11.11%
1025,NILUSD_720.csv,1742990400,1743422400,2025-03-26 12:00:00,2025-03-31 12:00:00,11,-10.00%
...,...,...,...,...,...,...,...
23,REPUSD_720.csv,1475582400,1743422400,2016-10-04 12:00:00,2025-03-31 12:00:00,5676,8.45%
20,ETCUSD_720.csv,1469577600,1743422400,2016-07-27 00:00:00,2025-03-31 12:00:00,6338,0.02%
8,ETHUSD_720.csv,1438948800,1743422400,2015-08-07 12:00:00,2025-03-31 12:00:00,7004,0.62%
3,LTCUSD_720.csv,1382616000,1743422400,2013-10-24 12:00:00,2025-03-31 12:00:00,7698,7.83%


### Filter Datasets

In [34]:
import pandas as pd

def filter_by_time(df, start_day=None, end_day=None, return_df=False):
    """
    Filter rows in `df` where:
    - `first_date` is on or before `start_day` (if provided)
    - `last_date` is on or after `end_day` (if provided)

    If time is not provided in dates, defaults to 00:00:00.
    
    Parameters:
        df (pd.DataFrame): DataFrame with 'first_date' and 'last_date' columns (as datetime or string)
        start_day (str): e.g. "2025-03-27" or "2025-03-27 14:30:00"
        end_day (str): e.g. "2025-03-31" or "2025-03-31 23:59:59"
        return_df (bool): Whether to return the filtered DataFrame

    Returns:
        int: number of matching rows
        pd.DataFrame (optional): filtered DataFrame
    """
    
    # Convert date columns to datetime
    df = df.copy()
    df['first_date'] = pd.to_datetime(df['first_date'])
    df['last_date'] = pd.to_datetime(df['last_date'])

    # Convert inputs to datetime objects, defaulting to 00:00:00 if time not provided
    if start_day:
        start_day = pd.to_datetime(start_day)
        if start_day.hour == 0 and start_day.minute == 0 and start_day.second == 0:
            start_day = start_day.replace(hour=0, minute=0, second=0)
    if end_day:
        end_day = pd.to_datetime(end_day)
        if end_day.hour == 0 and end_day.minute == 0 and end_day.second == 0:
            end_day = end_day.replace(hour=0, minute=0, second=0)

    # Apply filters
    if start_day and end_day:
        mask = (df['first_date'] <= start_day) & (df['last_date'] >= end_day)
    elif start_day:
        mask = df['first_date'] <= start_day
    elif end_day:
        mask = df['last_date'] >= end_day
    else:
        mask = pd.Series([True] * len(df))  # No filtering

    filtered_df = df[mask]
    
    if return_df:
        return len(filtered_df), filtered_df
    else:
        return len(filtered_df)

In [16]:
d20 = filter_by_time(df60, start_day="2020-01-01", end_day="2025-03-31")
d22 = filter_by_time(df60, start_day="2022-01-01", end_day="2025-03-31")
print("datasets available from 2020.1.1-2025.3.31: ", d20)
print("datasets available from 2022.1.1-2025.3.31: ", d22)

datasets available from 2020.1.1-2025.3.31:  93
datasets available from 2022.1.1-2025.3.31:  305


In [52]:


def compute_hours(start_day, end_day):
    """ 
    Calculate the number of hours elapsed between two dates.
    """
    start = pd.to_datetime(start_day)
    end = pd.to_datetime(end_day)
    hours = (end - start).total_seconds() / 3600
    return int(hours)


def compute_entries_in_range(data_input, start_date, end_date):
    """
    Count rows in dataset within the given date range.
    Args:
        data_input: Either a path to CSV file or a pandas DataFrame
        start_date: Start date in 'YYYY-MM-DD' or 'YYYY-MM-DD HH:MM:SS' format
        end_date: End date in 'YYYY-MM-DD' or 'YYYY-MM-DD HH:MM:SS' format
        (assume time is 00:00:00 if not given)
    Returns:
        int: Number of rows in range, or False if range is invalid
    """
    start_dt = pd.to_datetime(start_date).replace(hour=0, minute=0, second=0, microsecond=0) if len(start_date) <= 10 else pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date).replace(hour=0, minute=0, second=0, microsecond=0) if len(end_date) <= 10 else pd.to_datetime(end_date)
    start_unix, end_unix = start_dt.timestamp(), end_dt.timestamp()

    # Handle input data
    df = pd.read_csv(data_input) if isinstance(data_input, str) else data_input
    
    if df.iloc[:, 0].min() > start_unix or df.iloc[:, 0].max() < end_unix:
        return False
        
    # Count rows in range (inclusive)
    mask = (df.iloc[:, 0] >= start_unix) & (df.iloc[:, 0] <= end_unix)
    return len(df[mask])


# Get filtered dataframe and sort by number_rows
start_day = "2022-01-01"
end_day = "2025-03-31"

filtered_len, filtered_df = filter_by_time(df60, start_day=start_day, end_day=end_day, return_df=True)
print("Length of filtered dataframe: ", filtered_len)
# Test the function with the date range from the previous example
expected_rows = compute_hours(start_day, end_day)
print(f"Expected number of rows: {expected_rows} hours")


Length of filtered dataframe:  305
Expected number of rows: 28440 hours


In [1]:
import pandas as pd
def filter_by_suffix(name, df, return_df=False):
    """
    Filter rows where file_name contains the currency "name" right before the underscore.
    e.g. name="USD" then return files with names like "BTCUSD_60.csv" but not "USDEUR_60.csv"

    Args:
        name (str): The currency name to search for (e.g., 'USD', 'USDT')
        df (pd.DataFrame): The dataframe to filter, with file_name column
        return_df (bool): If True, return filtered dataframe along with count
        
    Returns:
        tuple: (count, filtered_df) if return_df=True, else just count
    """
    # Create pattern that matches prefix right before underscore
    pattern = f"{name}_"
    
    # Filter rows where file_name contains the pattern
    mask = df['file_name'].str.contains(pattern, regex=False)
    filtered_df = df[mask]
    
    if return_df:
        return len(filtered_df), filtered_df
    return len(filtered_df)

# Test the function
# test_prefixes = ['USD', 'USDT', 'EUR']
# for prefix in test_prefixes:
#     count, filtered = filter_by_suffix(prefix, filtered_df, return_df=True)
#     print(f"\nPrefix '{prefix}':")
#     print(f"Found {count} matching files")
#     print("First few matches:")
#     print(filtered['file_name'].head())

def find_missing_entries(csv_path, start_day, end_day, interval='60'):
    """
    Find missing entries in a time series dataset based on expected timesteps.
    
    Args:
        csv_path (str): Path to the CSV file
        start_day (str): Start date in format "YYYY-MM-DD" or "YYYY-MM-DD HH:MM:SS"
        end_day (str): End date in format "YYYY-MM-DD" or "YYYY-MM-DD HH:MM:SS"
        interval (str): Time interval in minutes (default '60')
        
    Returns:
        List of missing entry indices (0-based)
    """
    # Convert interval to seconds
    timestep = int(interval) * 60
    
    # Read the CSV file
    df = pd.read_csv(csv_path, header=None)
    
    # Convert start and end days to unix timestamps
    start_unix = pd.to_datetime(start_day).timestamp()
    end_unix = pd.to_datetime(end_day).timestamp()
    
    # Filter data within date range
    mask = (df[0] >= start_unix) & (df[0] <= end_unix)
    df = df[mask]
    
    # Convert timestamps to indices (0-based) first
    actual_entries = ((df[0] - start_unix) // timestep).astype(int)
    
    # Calculate expected entries and create set of expected indices
    expected_entries = int((end_unix - start_unix)/timestep) + 1
    # Find missing indices
    missing_indices = set(range(expected_entries)) - set(actual_entries)
    
    return sorted(missing_indices)


In [5]:
pd.set_option('display.width', 1000)
csv = 'Kraken_OHLCVT_summary/csv_file_summary.csv_60.csv'
summary = pd.read_csv(csv)
num, dfUSD = filter_by_suffix("USD", summary, return_df=True)
print(num)
dfUSD = dfUSD.reset_index(drop=True)
print(dfUSD.tail(10))

390
            file_name  first_unix_time  last_unix_time           first_date            last_date  number_rows
380   LSETHUSD_60.csv       1742223600      1743264000  2025-03-17 15:00:00  2025-03-29 16:00:00          121
381    DUCKUSD_60.csv       1742396400      1743462000  2025-03-19 15:00:00  2025-03-31 23:00:00          225
382  APENFTUSD_60.csv       1742482800      1743462000  2025-03-20 15:00:00  2025-03-31 23:00:00          271
383   SPICEUSD_60.csv       1742482800      1743462000  2025-03-20 15:00:00  2025-03-31 23:00:00          231
384     NILUSD_60.csv       1742997600      1743462000  2025-03-26 14:00:00  2025-03-31 23:00:00          123
385     WALUSD_60.csv       1743069600      1743462000  2025-03-27 10:00:00  2025-03-31 23:00:00          110
386    TERMUSD_60.csv       1743076800      1743447600  2025-03-27 12:00:00  2025-03-31 19:00:00           37
387  GHIBLIUSD_60.csv       1743098400      1743462000  2025-03-27 18:00:00  2025-03-31 23:00:00          102
388   

In [65]:
N, df_USD = filter_by_suffix("USD", filtered_df, return_df=True)
print(N)

97


In [71]:
k = 0
sample_file = f"Kraken_OHLCVT/{df_USD.iloc[k]['file_name']}"
list = find_missing_entries(sample_file, start_day, end_day)
print(list[:100])
print(len(list))

[0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 21, 22, 24, 25, 26, 28, 31, 32, 34, 36, 37, 38, 39, 40, 41, 42, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 64, 66, 67, 72, 74, 75, 76, 78, 79, 80, 81, 84, 85, 88, 89, 96, 98, 100, 101, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 118, 119, 120, 121, 130, 131, 132, 133, 134, 135, 137, 138, 139, 143, 144, 145, 146, 149, 151, 152, 154, 155, 156]
21686


#### Working with saved `filtered_df` 
(60min data from 2020-01-01 00:00:00 to 2-25-03-31 00:00:00, with 305 tokens available)

In [ ]:
import numpy as np

# Create a list to store results
results = []
#missing = [] # a list of lists containing missing entries for each file

# Calculate entries for each file
for k in range(N):
    sample_file = f"Kraken_OHLCVT/{df_USD.iloc[k]['file_name']}"
    #missing.append(find_missing_entries(sample_file, start_day, end_day))
    results.append(compute_entries_in_range(sample_file, start_day, end_day))

# Convert to numpy array and get sorted indices
results_array = np.array(results)
sorted_indices = np.argsort(results_array)

# Sort both results and filtered_df using the same indices
sorted_results = results_array[sorted_indices]
df_USD = df_USD.iloc[sorted_indices].reset_index(drop=True)


In [75]:
# Print sorted results
for i, (file_name, result) in enumerate(zip(df_USD['file_name'], sorted_results)):
    print(f"{i+1:4d} {file_name:20s}: {result}/{expected_rows} existing entries")

   1 TBTCUSD_60.csv      : 6755/28440 existing entries
   2 BNTUSD_60.csv       : 13585/28440 existing entries
   3 REPUSD_60.csv       : 13649/28440 existing entries
   4 GHSTUSD_60.csv      : 14132/28440 existing entries
   5 SDNUSD_60.csv       : 14766/28440 existing entries
   6 BADGERUSD_60.csv    : 15864/28440 existing entries
   7 PERPUSD_60.csv      : 16434/28440 existing entries
   8 OXYUSD_60.csv       : 16505/28440 existing entries
   9 WBTCUSD_60.csv      : 16658/28440 existing entries
  10 BALUSD_60.csv       : 17054/28440 existing entries
  11 RAYUSD_60.csv       : 18940/28440 existing entries
  12 CTSIUSD_60.csv      : 19178/28440 existing entries
  13 KARUSD_60.csv       : 19403/28440 existing entries
  14 BNCUSD_60.csv       : 19565/28440 existing entries
  15 BANDUSD_60.csv      : 19661/28440 existing entries
  16 KEEPUSD_60.csv      : 20161/28440 existing entries
  17 OGNUSD_60.csv       : 20236/28440 existing entries
  18 LSKUSD_60.csv       : 20367/28440 existing e

df_USD.iloc[1:].to_csv('USD_60_2022_01_01-2025_03_31.csv', index=False)

`USD_60_2022_01_01-2025_03_31.csv`: datasets with 60min, paired with USD, with over 13585/28440 existing timesteps from 2022-01-01 to 2025-03-31 (last date from Kraken)

In [ ]:
import pandas as pd
import h5py

def csv_to_h5(summary_csv, folder='Kraken_OHLCVT', savepath=None):
    """
    Given a summary CSV path or DataFrame with a 'file_name' column,
    saves all the CSVs listed in 'file_name' as datasets in an HDF5 file.

    Parameters:
        summary_csv: str or pd.DataFrame
            Path to the summary CSV or a DataFrame containing 'file_name' column.
        folder: str
            Parent folder containing the CSV files. Default is 'Kraken_OHLCVT'.
        savepath: str or None
            Path to the output HDF5 file. If None:
                - If summary_csv is a CSV file, use the same path with .h5 extension.
                - Otherwise, use 'selected_df.h5'.
    """

    import os

    # Load summary DataFrame if a path is given
    if isinstance(summary_csv, str):
        summary_df = pd.read_csv(summary_csv)
        if savepath is None:
            base, ext = os.path.splitext(summary_csv)
            savepath = base + '.h5'
    else:
        summary_df = summary_csv
        if savepath is None:
            savepath = 'selected_df.h5'

    file_names = summary_df['file_name'].tolist()

    with h5py.File(savepath, 'w') as h5f:
        for idx, fname in enumerate(file_names):
            csv_path = os.path.join(folder, fname) if folder else fname
            df = pd.read_csv(csv_path)
            dset_name = str(idx)
            if all([pd.api.types.is_numeric_dtype(df[col]) for col in df.columns]):
                h5f.create_dataset(dset_name, data=df.values)
            else:
                data_as_str = df.astype(str).values.astype('S')
                h5f.create_dataset(dset_name, data=data_as_str)
            h5f[dset_name].attrs['file_name'] = fname
# Example usage:
# csv_to_h5('/Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/USD_60_2022_01_01-2025_03_31.csv')


# Path to your HDF5 file (change as needed)


datasets in an HDF5 file are not inherently ordered — they are accessed like items in a dictionary, by their key (name)

In [82]:
h5path = '/Users/xin/Stanford/MSE 349 Financial Statistics/crypto_data/USD_60_2022_01_01-2025_03_31.h5'
with h5py.File(h5path, 'r') as h5f:
    print("Datasets in this HDF5 file:")
    for dset_name in h5f:
        file_name = h5f[dset_name].attrs.get('file_name', None)
        print(f"Dataset: {dset_name}, file_name: {file_name}")

Datasets in this HDF5 file:
Dataset: 0, file_name: BNTUSD_60.csv
Dataset: 1, file_name: REPUSD_60.csv
Dataset: 10, file_name: CTSIUSD_60.csv
Dataset: 11, file_name: KARUSD_60.csv
Dataset: 12, file_name: BNCUSD_60.csv
Dataset: 13, file_name: BANDUSD_60.csv
Dataset: 14, file_name: KEEPUSD_60.csv
Dataset: 15, file_name: OGNUSD_60.csv
Dataset: 16, file_name: LSKUSD_60.csv
Dataset: 17, file_name: REPV2USD_60.csv
Dataset: 18, file_name: GNOUSD_60.csv
Dataset: 19, file_name: KNCUSD_60.csv
Dataset: 2, file_name: GHSTUSD_60.csv
Dataset: 20, file_name: MLNUSD_60.csv
Dataset: 21, file_name: ICXUSD_60.csv
Dataset: 22, file_name: RARIUSD_60.csv
Dataset: 23, file_name: YFIUSD_60.csv
Dataset: 24, file_name: MIRUSD_60.csv
Dataset: 25, file_name: LPTUSD_60.csv
Dataset: 26, file_name: CQTUSD_60.csv
Dataset: 27, file_name: KILTUSD_60.csv
Dataset: 28, file_name: RENUSD_60.csv
Dataset: 29, file_name: QTUMUSD_60.csv
Dataset: 3, file_name: SDNUSD_60.csv
Dataset: 30, file_name: OXTUSD_60.csv
Dataset: 31, file

In [2]:
import pandas as pd

def add_missing_row_info(summary_csv, timestep=3600):
    """
    Adds number_missing and ratio_missing columns to the summary CSV file,
    assuming uniform time intervals between first_unix_time and last_unix_time.
    """
    df = pd.read_csv(summary_csv)

    # Compute expected rows from the time range and timestep
    df['expected_rows'] = (df['last_unix_time'] - df['first_unix_time']) // timestep + 1
    missing_rows = df['expected_rows'] - df['number_rows']
    df['ratio_missing'] = missing_rows / df['expected_rows']

    # Drop helper column if not needed
    df.drop(columns=['expected_rows'], inplace=True)

    # Overwrite the original file
    #df.to_csv(summary_csv, index=False)
    return df


In [3]:
df_summary = add_missing_row_info('USD_60_2022_01_01-2025_03_31_summary.csv')

In [4]:
df_summary.head(10)

,file_name,first_unix_time,last_unix_time,first_date,last_date,number_rows,ratio_missing
0,BNTUSD_60.csv,1621868400,1743451200,2021-05-24 15:00:00,2025-03-31 20:00:00,16000,0.526263
1,REPUSD_60.csv,1475611200,1743458400,2016-10-04 20:00:00,2025-03-31 22:00:00,45977,0.382054
2,GHSTUSD_60.csv,1621263600,1743462000,2021-05-17 15:00:00,2025-03-31 23:00:00,17687,0.478951
3,SDNUSD_60.csv,1630594800,1743462000,2021-09-02 15:00:00,2025-03-31 23:00:00,17614,0.438204
4,BADGERUSD_60.csv,1628002800,1743451200,2021-08-03 15:00:00,2025-03-31 20:00:00,18982,0.408107
5,PERPUSD_60.csv,1626188400,1743462000,2021-07-13 15:00:00,2025-03-31 23:00:00,19777,0.392915
6,OXYUSD_60.csv,1632841200,1743458400,2021-09-28 15:00:00,2025-03-31 22:00:00,18458,0.399310
7,WBTCUSD_60.csv,1628010000,1743462000,2021-08-03 17:00:00,2025-03-31 23:00:00,17874,0.442674
8,BALUSD_60.csv,1600351200,1743433200,2020-09-17 14:00:00,2025-03-31 15:00:00,27349,0.311906
9,RAYUSD_60.csv,1632841200,1743462000,2021-09-28 15:00:00,2025-03-31 23:00:00,20742,0.325002


### Process CSV Files & Compute Log Returns

- Add column names
- Truncate to desired timeframe
- Add index (in 0,1,2,...)
- Find missing indices
- Add log returns

In [ ]:
import pandas as pd
import numpy as np

def add_colnames(input_data, column_names=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'trades']):
    """
    Add column names to a DataFrame without a header, from a CSV file or an existing DataFrame.
    input_data: String (CSV file path) or pandas DataFrame.
    """
    # Check if input_data is a string (CSV path)
    if isinstance(input_data, str):
        df = pd.read_csv(input_data, header=None)
    # Check if input_data is a DataFrame
    elif isinstance(input_data, pd.DataFrame):
        df = input_data.copy()  # Avoid modifying the original
    # Validate number of columns
    num_cols = df.shape[1]
    num_expected = len(column_names)
    
    if num_cols != num_expected:
        raise ValueError(f"DataFrame has {num_cols} columns, but {num_expected} column names were provided: {column_names}")
    df.columns = column_names
    return df

def add_index_truncate(df, start_time, end_time, timestep):
    """
    Adds an 'index' column (in range(T)) 
    Truncates the df to only keep the range [start_time-timestep, end_time] # keep -1 for return computation
    Return the df with a list of missing indices
    """
    df = df.copy()
    # Truncate to only keep rows within [start_time-timestep, end_time] (need -1 entry to compute return)
    df = df[(df['timestamp'] >= start_time-timestep) & (df['timestamp'] <= end_time)].reset_index(drop=True)
    df['index'] = ((df['timestamp'] - start_time) // timestep).astype(int)
    T = int((end_time - start_time) // timestep) + 1
    missing_indices = sorted(list(set(range(T)) - set(df[1:]['index'])))
    return df, missing_indices


def add_log_return(df):
    """
    Input: df with 'index' and 'close' columns.
    Add a 'log_return' column to the df
    Delete index -1 to have time within [start_time, end_time]
    """
    df = df.copy()
    df['log_return'] = np.nan

    if df.loc[0,'index'] == -1:
        if not np.isnan(df.loc[0,'close']) and not np.isnan(df.loc[1,'close']):
            df.loc[1,'log_return'] = np.log(df.loc[0,'close'] / df.loc[1,'close'])
        # Remove the first row and reset index
        df = df.iloc[1:].reset_index(drop=True)

    for i in range(1, len(df)):
        prev_close = df.loc[i-1, 'close']
        curr_close = df.loc[i, 'close']
        if df.loc[i,'index'] == df.loc[i-1,'index'] + 1 and not np.isnan(prev_close) and not np.isnan(curr_close):
            df.loc[i,'log_return'] = np.log(curr_close / prev_close)
        else:
            df.loc[i,'log_return'] = np.nan
    return df

Testing the functions

In [8]:
# # Set timeframe
start_day="2022-01-01"
end_day="2025-03-31"
start_unix = int(pd.to_datetime(start_day).timestamp())
end_unix = int(pd.to_datetime(end_day).timestamp())
print(start_unix, end_unix)
print(pd.to_datetime(start_unix, unit='s'), pd.to_datetime(end_unix, unit='s'))

1640995200 1743379200
2022-01-01 00:00:00 2025-03-31 00:00:00


Truncate the datasets, add headers, index column, log returns, and save them to a new h5 file indexed by 0,1...N, save missing indices (list of lists) to pickle file

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.width', 1000) 
df = pd.read_csv('USD_60_2022_01_01-2025_03_31.csv')
filepaths = [f'Kraken_OHLCVT/{filename}' for filename in df['file_name']]

# # Save filepaths to a text file in a readable format
# savepath = 'USD_60_2022_01_01-2025_03_31_filenames.txt'
# with open(savepath, 'w') as f:
#     for path in filepaths:
#         f.write(f"{path}\n")
# # Read the file and convert each line into a string in a list
# with open("USD_60_2022_01_01-2025_03_31_filenames.txt", "r") as file:
#     filepaths = [line.strip() for line in file if line.strip()]

In [ ]:
df2 = pd.read_csv(filepaths[1], header=None)
df2 = add_colnames(df2)
df22, missing2 = add_index_truncate(df2, start_unix, end_unix, 3600)
df22 = add_log_return(df22)
print(df22.head(10))

    timestamp    open    high     low   close       volume  trades  index  log_return
0  1640995200  18.183  18.950  18.183  18.768   472.512747      39      0   -0.032657
1  1640998800  18.870  18.870  18.870  18.870    51.825791       2      1    0.005420
2  1641002400  19.191  19.229  19.047  19.047   424.269850      38      2    0.009336
3  1641006000  19.251  19.453  19.229  19.366   224.133773      21      3    0.016609
4  1641009600  19.381  19.509  19.014  19.200  1805.804939      37      4   -0.008609
5  1641013200  19.200  19.283  18.848  19.000    41.684101       8      5   -0.010471
6  1641027600  18.770  18.790  18.705  18.705     4.540137       6      9         NaN
7  1641031200  18.778  18.778  18.689  18.689   354.129610      12     10   -0.000856
8  1641056400  18.766  18.766  18.766  18.766     2.618940       1     17         NaN
9  1641060000  18.698  18.698  18.698  18.698   235.519170       2     18   -0.003630


In [ ]:
# !pip install pyarrow # pyarrow is required for parquet support
# !pip install fastparquet

In [20]:
# Truncate datasets in filepaths to desired timeframe and save them as parquet files to output_dir
# parquet is the most efficient way of saving dfs, it's much faster to process than csv or h5 and takes less storage

import pandas as pd
import os

output_dir = "USD_60_2022-2025"
os.makedirs(output_dir, exist_ok=True)
list_missing = []
for fp in filepaths:
    stock_name = os.path.splitext(os.path.basename(fp))[0]  # e.g., "AAPL"
    df = pd.read_csv(fp)
    df = add_colnames(df)
    df, missing = add_index_truncate(df, start_unix, end_unix, 3600)
    df = add_log_return(df)
    list_missing.append(missing)
    df.to_parquet(f"{output_dir}/{stock_name}.parquet", index=False)

# load parquet file
# dfp = pd.read_parquet(path)

In [ ]:
def dropna_truncate(df, start_time, end_time, timestep):
    """
    dropna and truncates the df to only keep the range [start_time-timestep, end_time], and reset index
    """
    df = df[(df['timestamp'] >= start_time-timestep) & (df['timestamp'] <= end_time)].reset_index()
    df['index'] = ((df['timestamp'] - start_time) // timestep).astype(int)
    df.set_index('index', inplace=True)
    # T = int((end_time - start_time) // timestep) + 1
    # missing_indices = sorted(list(set(range(T)) - set(df[1:]['index'])))
    return df

In [3]:
with open('filenames_parquet.txt', 'r') as f:
        filepaths = f.read().splitlines()

filepaths = ['USD_60_betas/' + f for f in filepaths]


['/USD_60_indicators/BNTUSD_60.parquet', '/USD_60_indicators/REPUSD_60.parquet', '/USD_60_indicators/CTSIUSD_60.parquet', '/USD_60_indicators/KARUSD_60.parquet', '/USD_60_indicators/BNCUSD_60.parquet', '/USD_60_indicators/BANDUSD_60.parquet', '/USD_60_indicators/KEEPUSD_60.parquet', '/USD_60_indicators/OGNUSD_60.parquet', '/USD_60_indicators/LSKUSD_60.parquet', '/USD_60_indicators/REPV2USD_60.parquet', '/USD_60_indicators/GNOUSD_60.parquet', '/USD_60_indicators/KNCUSD_60.parquet', '/USD_60_indicators/GHSTUSD_60.parquet', '/USD_60_indicators/MLNUSD_60.parquet', '/USD_60_indicators/ICXUSD_60.parquet', '/USD_60_indicators/RARIUSD_60.parquet', '/USD_60_indicators/YFIUSD_60.parquet', '/USD_60_indicators/MIRUSD_60.parquet', '/USD_60_indicators/LPTUSD_60.parquet', '/USD_60_indicators/CQTUSD_60.parquet', '/USD_60_indicators/KILTUSD_60.parquet', '/USD_60_indicators/RENUSD_60.parquet', '/USD_60_indicators/QTUMUSD_60.parquet', '/USD_60_indicators/SDNUSD_60.parquet', '/USD_60_indicators/OXTUSD_60.

In [16]:
import pandas as pd
A = pd.read_parquet('USD_60_2022/ETCUSD_60.parquet')
print("Column names in the dataframe:")
for i, col in enumerate(A.columns, 1):
    print(f"{i}. {col}")



Column names in the dataframe:
1. timestamp
2. open
3. high
4. low
5. close
6. volume
7. trades
8. log_return
9. return
10. volume_change
11. close_to_high
12. close_to_low
13. log_price_range
14. ma_zscore_12
15. mom_24_4
16. mom_168_24
17. strev_24
18. ltrev_168
19. ema_diff_norm_12_48
20. ema_diff_norm_24_120
21. price_ema_diff_12
22. macd_hist
23. realized_vol_24
24. weighted_vol_24
25. volume_zscore_24
26. vwap_24
27. sto_osc_12
28. rsi
29. amihud_12


In [17]:
A.head(20)
print(len(A))
print(len(A.dropna()))

26599
26599


28412